In [27]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [3,141 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [1,369 kB]
Err:13 http://archive.ubuntu.com/ubuntu bionic InRelease
  Could not connect 

In [28]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-ChallengeVineReviews").getOrCreate()

In [29]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [30]:
clean = df.dropna()
clean.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12190288|R3FU16928EP5TC|B00AYB1482|     668895143|Enlightened: Seas...|Digital_Video_Dow...|          5|            0|          0|   N|                Y|I loved it and I ...|I loved it and I ...| 2015-08-31|
|         US|   30549954|R1IZHHS1MH3AQ4|B00KQD28OM|     246219280|             Vicious|Digital_Video_Dow

In [31]:

# Create the vine_table. DataFrame
vine_table = clean.select(["review_id","star_rating", "helpful_votes","total_votes","vine","verified_purchase"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3FU16928EP5TC|          5|            0|          0|   N|                Y|
|R1IZHHS1MH3AQ4|          5|            0|          0|   N|                Y|
| R52R85WC6TIAH|          4|           17|         18|   N|                Y|
| R7HOOYTVIB0DS|          5|            0|          0|   N|                Y|
|R1XQ2N5CDOZGNX|          5|            0|          0|   N|                Y|
|R1HCST57W334KN|          5|            5|          6|   N|                Y|
|R32BUTYQS1ZJBQ|          4|            1|          1|   N|                Y|
|  RH4SXPL4L9QU|          3|            0|          0|   N|                Y|
|R37INWIQA5YW8N|          2|            0|          0|   N|                Y|
|R18GSVAAS3N8GX|          3|            0|          0|   N|     

In [45]:

# Step 1: Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
from pyspark.sql.functions import col
total_vote = vine_table.filter(col("total_votes") >= 20)
total_vote.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1CY3APKBOVUYU|          1|            9|         33|   N|                Y|
|R2LTI28AN2281A|          1|            0|         22|   N|                Y|
|R3PENXZQNYN29L|          1|            1|         26|   N|                Y|
| R8IGXNWIBN7ZP|          1|            6|         20|   N|                Y|
|R2P8B83Q61L4HS|          1|            0|         22|   N|                Y|
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|
|R257YW4XQLDH1M|          1|            6|         20|   N|                Y|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|
| RGCY6WG37F810|          4|           24|         27|   N|                N|
|R1TIDPK91S2IRQ|          1|            0|         25|   N|     

In [33]:
# Step 2: Retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
percent_votes = total_vote.withColumn('percent_votes',col('helpful_votes')/col('total_votes')).filter(col("percent_votes") >= 0.5)
percent_votes.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|0.7804878048780488|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|0.8260869565217391|
| RGCY6WG37F810|          4|           24|         27|   N|                N|0.8888888888888888|
|R2UH0VN09DDQLA|          3|           23|         30|   N|                N|0.7666666666666667|
|R1E4W9ROP10WT3|          5|           19|         22|   N|                N|0.8636363636363636|
| RF2BLXZTUOL0M|          5|           23|         27|   N|                N|0.8518518518518519|
| REMDS9V4J58OR|          4|           25|         29|   N|                Y|0.8620689655172413|
|R34CAWTWR5O2VN|          1|  

In [34]:
# Step 3: Retrieves all the rows where a review was written as part of the Vine program (paid)
vine_program = percent_votes.filter(col("vine") == "Y").show(10)

+---------+-----------+-------------+-----------+----+-----------------+-------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percent_votes|
+---------+-----------+-------------+-----------+----+-----------------+-------------+
+---------+-----------+-------------+-----------+----+-----------------+-------------+



In [43]:
# Step 4: Retrieves all the rows where the review was not part of the Vine program (unpaid)
non_vine_program = percent_votes.filter(col("vine") == "N").show(10)
    

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|     percent_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RZM83AGJAUZMA|          1|           32|         41|   N|                Y|0.7804878048780488|
|R12MYB9QM2YN8B|          1|           19|         23|   N|                N|0.8260869565217391|
| RGCY6WG37F810|          4|           24|         27|   N|                N|0.8888888888888888|
|R2UH0VN09DDQLA|          3|           23|         30|   N|                N|0.7666666666666667|
|R1E4W9ROP10WT3|          5|           19|         22|   N|                N|0.8636363636363636|
| RF2BLXZTUOL0M|          5|           23|         27|   N|                N|0.8518518518518519|
| REMDS9V4J58OR|          4|           25|         29|   N|                Y|0.8620689655172413|
|R34CAWTWR5O2VN|          1|  

In [41]:
nvp_count = non_vine_program.count()
nvp_count

11362

In [44]:
# Step 5: Determine the total number of reviews, the number of 5-star reviews, and the percentage of 5-star reviews for the two types of review (paid vs unpaid).

from pyspark.sql.functions import col,when,count,lit
ratings_total = percent_votes.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total_5_Star_Reviews"),
        (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("5_Star_%")).show()

+----+-------------+--------------------+-----------------+
|vine|Total_Reviews|Total_5_Star_Reviews|         5_Star_%|
+----+-------------+--------------------+-----------------+
|   N|        11362|                4670|41.10191867628939|
+----+-------------+--------------------+-----------------+

